In [2]:
import tensorflow as tf
import numpy as np

In [7]:
model = tf.keras.Sequential([tf.keras.layers.Dense(32)])

In [4]:
model.build([None, 3])
print(model)

In [15]:
opt = tf.keras.optimizers.SGD(0.1)
baselogger = tf.keras.callbacks.ReduceLROnPlateau()
model.callbacks= [baselogger]
model.optimizer = opt
print(model.callbacks[0])

In [17]:
for i in None:
    print(i)

TypeError: 'NoneType' object is not iterable

In [46]:
import tensorflow as tf
import time
import numpy as np
import os

N = 64
C = 256
G = 32
H, W = 64, 64
print("N, C, H, W:", [N, C, H, W])


def benchmark_all(use_loop, format):
    shape4d = [N, C, H, W] if format == 'NCHW' else [N, H, W, C]

    input = tf.Variable(np.random.rand(*shape4d), dtype=tf.float32)
    filter_shape = [3, 3, C // G, C]
    filter = tf.Variable(np.random.rand(*filter_shape), dtype=tf.float32)
    print(input.shape, filter.shape)

    @tf.function
    def forward(use_loop):
        if use_loop:
            inputs = tf.split(input, G, axis=1 if format == 'NCHW' else 3)
            filters = tf.split(filter, G, axis=3)
            output = tf.concat(
                [tf.nn.conv2d(i, f,
                    strides=[1,1,1,1],
                    padding='SAME',
                    data_format=format) for i, f in zip(inputs, filters)], axis=1 if format == 'NCHW' else 3)
        else:
            output = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME', data_format=format)
        cost = tf.reduce_sum(output)
        return cost
    
    optim = tf.keras.optimizers.SGD(0.1)
    
    @tf.function
    def forward_backward(use_loop):
        with tf.GradientTape() as tape:
            cost = forward(use_loop)
            gradients = tape.gradient(cost, filter)
            optim.apply_gradients([[gradients, filter]])

    def benchmark(op, nr_iter=200, nr_warmup=10):
        for k in range(nr_warmup):
            op(use_loop)
        start = time.perf_counter()
        for k in range(nr_iter):
            op(use_loop)
        end = time.perf_counter()
        itr_per_sec = nr_iter * 1. / (end - start)
        return itr_per_sec



    spd_forward = benchmark(forward)
    print("Loop={}, Format={}, Forward: {} itr/s".format(use_loop, format, spd_forward))
    spd_backward = benchmark(forward_backward)
    print("Loop={}, Format={},  Forward-Backward: {} itr/s".format(use_loop, format, spd_backward))


formats = ['NHWC', 'NCHW']
for format in formats:
    for use_loop in [True, False]:
        benchmark_all(use_loop, format)

N, C, H, W: [64, 256, 64, 64]
(64, 64, 64, 256) (3, 3, 8, 256)
Loop=True, Format=NHWC, Forward: 84.15945928093085 itr/s
Loop=True, Format=NHWC,  Forward-Backward: 55.92312216237774 itr/s
(64, 64, 64, 256) (3, 3, 8, 256)
Loop=False, Format=NHWC, Forward: 156.84641150557434 itr/s
Loop=False, Format=NHWC,  Forward-Backward: 98.38231233412873 itr/s
(64, 256, 64, 64) (3, 3, 8, 256)
Loop=True, Format=NCHW, Forward: 104.09903200573638 itr/s
Loop=True, Format=NCHW,  Forward-Backward: 63.83993698434464 itr/s
(64, 256, 64, 64) (3, 3, 8, 256)
Loop=False, Format=NCHW, Forward: 188.0797542247718 itr/s
Loop=False, Format=NCHW,  Forward-Backward: 118.61153408803331 itr/s
